In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.models import load_model
from keras.layers import Input, Dense, Flatten, Reshape, Conv1D, Conv1DTranspose, BatchNormalization, Cropping1D, MaxPooling1D, Dropout
from keras.optimizers import RMSprop, Adam, Nadam
from keras.callbacks import EarlyStopping
from keras import backend as K
from tensorflow import keras
import matplotlib.pyplot as plt # plotting
from sklearn.ensemble import RandomForestClassifier # randomForest
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import sys

from utili import boxlist
np.set_printoptions(threshold=sys.maxsize)

In [2]:
import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter

In [3]:
overlap = False
if overlap:
    X = pd.read_csv('data/X_o.csv', header=None)
    y = pd.read_csv('data/y_o.csv', header = None)
else:
    X = pd.read_csv('data/X_New.csv', header=None)
    y = pd.read_csv('data/y_New.csv', header = None)
y = y > 10
y = y * 1
print(X.shape)
print(y.shape)

(10068, 20)
(10068, 1)


In [4]:
sum(y==1)

0

In [5]:
print(y.to_numpy().flatten())

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [6]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.001144,0.005588,0.006636,-0.015492,-0.055692,-0.051978,0.039127,0.142835,0.126274,-0.027424,-0.162175,-0.122227,0.043465,0.134283,0.047762,-0.092512,-0.099568,0.023747,0.094899,0.007221
1,-0.162175,-0.122227,0.043465,0.134283,0.047762,-0.092512,-0.099568,0.023747,0.094899,0.007221,-0.125459,-0.119523,0.032543,0.144700,0.072357,-0.096765,-0.148476,-0.023332,0.106300,0.072732
2,-0.125459,-0.119523,0.032543,0.144700,0.072357,-0.096765,-0.148476,-0.023332,0.106300,0.072732,-0.056630,-0.076536,0.059737,0.171569,0.105960,-0.043815,-0.066033,0.069669,0.176758,0.121341
3,-0.056630,-0.076536,0.059737,0.171569,0.105960,-0.043815,-0.066033,0.069669,0.176758,0.121341,-0.002463,-0.029760,0.029883,0.023307,-0.092713,-0.167076,-0.081010,0.056880,0.054975,-0.089383
4,-0.002463,-0.029760,0.029883,0.023307,-0.092713,-0.167076,-0.081010,0.056880,0.054975,-0.089383,-0.189468,-0.133676,-0.030717,-0.038357,-0.131577,-0.142828,-0.013142,0.113058,0.084420,-0.047462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10063,0.031650,0.115371,-0.065914,-0.204990,0.001074,0.361403,0.371917,-0.101739,-0.521710,-0.302037,0.375157,0.730611,0.388696,-0.161821,-0.256278,0.091458,0.305501,0.099037,-0.161404,-0.057840
10064,0.375157,0.730611,0.388696,-0.161821,-0.256278,0.091458,0.305501,0.099037,-0.161404,-0.057840,0.240746,0.238133,-0.148583,-0.457552,-0.329220,0.003499,0.069469,-0.171392,-0.316017,-0.134737
10065,0.240746,0.238133,-0.148583,-0.457552,-0.329220,0.003499,0.069469,-0.171392,-0.316017,-0.134737,0.119123,0.115684,-0.085129,-0.171503,-0.040839,0.102595,0.081828,-0.016104,-0.016462,0.062313
10066,0.119123,0.115684,-0.085129,-0.171503,-0.040839,0.102595,0.081828,-0.016104,-0.016462,0.062313,0.065042,-0.029620,-0.078636,-0.003335,0.087571,0.043260,-0.125116,-0.249884,-0.193167,0.025415


In [7]:
from scipy.stats import skew


X = X.to_numpy()


X_features = pd.DataFrame({
    'mean': abs(X).mean(axis=1),
    'std': X.std(axis=1),
    'skew': skew(X, axis=1),
})

X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=22, shuffle=False)

In [8]:
print(X_features)

           mean       std      skew
0      0.065002  0.082517 -0.060250
1      0.088502  0.098775 -0.055197
2      0.094512  0.102247 -0.079885
3      0.078775  0.091839  0.166369
4      0.077608  0.084164  0.009906
...         ...       ...       ...
10063  0.235280  0.292222  0.317112
10064  0.236857  0.288016  0.572848
10065  0.146046  0.181352 -0.509665
10066  0.085631  0.104815 -0.637346
10067  0.139646  0.167607  0.113550

[10068 rows x 3 columns]


In [9]:
from sklearn.ensemble import GradientBoostingClassifier
# model = GradientBoostingClassifier(n_estimators=500, learning_rate=1.0, max_depth=3, random_state=0)

In [10]:
# Random Forest
model = RandomForestClassifier(random_state=15, max_depth=12, n_estimators=500)#,class_weight='balanced_subsample') # Random state for reproducibility (same results every time)
# classifier training
fit = model.fit(X_train, y_train)

In [11]:
# from sklearn import svm
# # model = svm.SVC(gamma=10)
# # model.fit(X_train, y_train)

# from sklearn import tree

# model = tree.DecisionTreeClassifier()
# model.fit(X_train, y_train)

In [12]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [13]:
def removeAlone1(p):
    if p[0]==1 and p[1]==0:
        p[0] = 0
    if p[-1]==1 and p[-2]==0:
        p[-1] = 0
    for icount, ivalue in enumerate(p):
        if ivalue==1:
            if p[icount-1]==0 and p[icount+1]==0:
                p[icount] = 0
    return p

In [14]:
def removeAlone0(p):
    if p[0]==0 and p[1]==1:
        p[0] = 1
    if p[-1]==0 and p[-2]==1:
        p[-1] = 1
    for icount, ivalue in enumerate(p):
        if ivalue==0:
            if p[icount-1]==1 and p[icount+1]==1:
                p[icount] = 1
    return p

In [15]:

print("----TRAIN RESULTS -------\n")
y_train_pred = model.predict(X_train)

y_test_pred = model.predict(X_test)

# print(y_pred)
# print(y_train.to_numpy().flatten())

----TRAIN RESULTS -------



In [16]:
y_train_pred = removeAlone1(y_train_pred)
y_test_pred = removeAlone1(y_test_pred)


# y_train_pred = removeAlone0(y_train_pred)
# y_test_pred = removeAlone0(y_test_pred)

In [17]:
y_trainBoxlist = boxlist.boxlist(y_test.to_numpy())
y_testBoxlist = boxlist.boxlist(y_test_pred)

In [18]:
test_event = y_trainBoxlist.shape[0]

pred_event = y_testBoxlist.shape[0]

print(test_event)
print(pred_event)

41
56


In [19]:
TP = 0
listTP = np.array([])
for i in y_testBoxlist:
    for j in y_trainBoxlist:
        if j[0]<=i[1] and j[1]>=i[0]:
            TP +=1 
            listTP = np.append(listTP, j[0])
np.unique(listTP).shape

(41,)

There are 41 events in total
predict 59 events, 51 event are overlap with true events, 8 events are false positive

In [20]:
TP

50

In [21]:
print("----TRAIN RESULTS -------\n")
print(classification_report(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))

print("\n\n----TEST RESULTS -------\n")
print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))

----TRAIN RESULTS -------

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7374
           1       0.99      0.91      0.95       680

    accuracy                           0.99      8054
   macro avg       0.99      0.95      0.97      8054
weighted avg       0.99      0.99      0.99      8054

[[7370    4]
 [  61  619]]


----TEST RESULTS -------

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1743
           1       0.85      0.77      0.81       271

    accuracy                           0.95      2014
   macro avg       0.91      0.88      0.89      2014
weighted avg       0.95      0.95      0.95      2014

[[1706   37]
 [  61  210]]


In [22]:
import pickle
with open('s1model.pkl','wb') as f:
    pickle.dump(model,f)

# # load
# with open('model.pkl', 'rb') as f:
#     model2 = pickle.load(f)


# y_train_pred = removeAlone1(model2.predict(X_test))
# print("\n\n----TEST RESULTS -------\n")
# print(classification_report(y_test, y_test_pred))
# print(confusion_matrix(y_test, y_test_pred))

In [23]:
loc = y.to_numpy().flatten().shape[0]-y_test_pred.shape[0]
print(y.to_numpy().flatten()[loc::])

[0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 0
 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0
 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [24]:
for depth in range(1,20):
    model = RandomForestClassifier(random_state=666, max_depth=depth, n_estimators=50)#,class_weight='balanced_subsample') # Random state for reproducibility (same results every time)
    # classifier training
    fit = model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    y_train_pred = removeAlone1(y_train_pred)
    y_test_pred = removeAlone1(y_test_pred)
    
    print("----", depth,"-----TRAIN RESULTS -------\n")
    print(classification_report(y_train, y_train_pred))
    print(confusion_matrix(y_train, y_train_pred))

    print("\n\n--", depth,"--TEST RESULTS -------\n")
    print(classification_report(y_test, y_test_pred))
    print(confusion_matrix(y_test, y_test_pred))
    
    
    print("---------------------------------------------------------------------------------")
    

---- 1 -----TRAIN RESULTS -------

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7374
           1       0.89      0.68      0.77       680

    accuracy                           0.97      8054
   macro avg       0.93      0.83      0.88      8054
weighted avg       0.96      0.97      0.96      8054

[[7318   56]
 [ 220  460]]


-- 1 --TEST RESULTS -------

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1743
           1       0.90      0.74      0.81       271

    accuracy                           0.95      2014
   macro avg       0.93      0.86      0.89      2014
weighted avg       0.95      0.95      0.95      2014

[[1721   22]
 [  71  200]]
---------------------------------------------------------------------------------
---- 2 -----TRAIN RESULTS -------

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7374


---- 11 -----TRAIN RESULTS -------

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7374
           1       0.97      0.87      0.92       680

    accuracy                           0.99      8054
   macro avg       0.98      0.94      0.96      8054
weighted avg       0.99      0.99      0.99      8054

[[7355   19]
 [  86  594]]


-- 11 --TEST RESULTS -------

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1743
           1       0.85      0.77      0.81       271

    accuracy                           0.95      2014
   macro avg       0.91      0.87      0.89      2014
weighted avg       0.95      0.95      0.95      2014

[[1707   36]
 [  63  208]]
---------------------------------------------------------------------------------
---- 12 -----TRAIN RESULTS -------

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      73